<a href="https://colab.research.google.com/github/Maynex69/Gestion-de-ingresos-hospitalarios/blob/main/Creaci%C3%B3n_de_nuevo_dataset_con_datos_m%C3%A1s_precisos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/dataset.csv', sep=',', header=0, index_col=None)

# Contar el número de veces que aparece cada condición médica
condition_counts = df['Medical Condition'].value_counts()

# Mostrar los resultados
print(condition_counts)

Medical Condition
Arthritis       9308
Diabetes        9304
Hypertension    9245
Obesity         9231
Cancer          9227
Asthma          9185
Name: count, dtype: int64


Reemplazando la cantidad de pacientes en nuestro Dataset por datos relativamente más reales, así como edades relativamente más coherentes.

In [20]:
# 1. Definir la distribución deseada para cada condición médica
desired_distribution = {
    'Diabetes': 0.21,
    'Arthritis': 0.16,
    'Obesity': 0.2,
    'Hypertension': 0.21,
    'Cancer': 0.08,
    'Asthma': 0.14
}

# 2. Contar el número total de pacientes
total_patients = len(df)

# 3. Calcular el número deseado de pacientes por condición
desired_counts = {cond: int(total_patients * perc) for cond, perc in desired_distribution.items()}

# 4. Crear un nuevo DataFrame con la redistribución deseada
new_df = pd.DataFrame()

for condition, desired_count in desired_counts.items():
    # Filtrar pacientes con la condición actual
    condition_df = df[df['Medical Condition'] == condition]

    # Si hay suficientes pacientes para la condición, muestrea o selecciona el número deseado
    if len(condition_df) > desired_count:
        condition_df = condition_df.sample(desired_count, random_state=42)  # Muestreo aleatorio
    else:
        # Si no hay suficientes pacientes, usa todos los disponibles
        condition_df = condition_df

    # Añadir al nuevo DataFrame
    new_df = pd.concat([new_df, condition_df], ignore_index=True)

# 5. Definir las edades y los porcentajes para cada condición
conditions = {
    'Diabetes': (60, 10, 0.8),
    'Arthritis': (75, 15, 0.9),
    'Obesity': (30, 20, 0.7),
    'Hypertension': (55, 15, 0.85),
    'Cancer': (65, 10, 0.95),
    'Asthma': (20, 15, 0.9)
}

for condition, (mean_age, std_dev, replace_percentage) in conditions.items():
    # Filtrar pacientes con la condición actual
    condition_df = new_df[new_df['Medical Condition'] == condition]

    # Calcular el número de pacientes a reemplazar
    num_to_replace = int(len(condition_df) * replace_percentage)

    # Seleccionar aleatoriamente pacientes para reemplazar
    indices_to_replace = np.random.choice(condition_df.index, num_to_replace, replace=False)

    # Generar edades basadas en una distribución normal
    new_ages = np.random.normal(mean_age, std_dev, num_to_replace).astype(int)

    # Aplicar los nuevos valores de edad
    new_df.loc[indices_to_replace, 'Age'] = new_ages

# Mostrar los resultados para verificar
print(new_df['Medical Condition'].value_counts())
print(new_df.groupby('Medical Condition')['Age'].describe())

Medical Condition
Diabetes        9304
Hypertension    9245
Obesity         9231
Arthritis       8880
Asthma          7770
Cancer          4440
Name: count, dtype: int64
                    count       mean        std   min   25%   50%   75%    max
Medical Condition                                                             
Arthritis          8880.0  74.519032  14.526993  21.0  65.0  75.0  84.0  130.0
Asthma             7770.0  19.786358  14.703328 -36.0  10.0  20.0  30.0   78.0
Cancer             4440.0  64.405856   9.999061  28.0  58.0  64.5  71.0  100.0
Diabetes           9304.0  59.405847  10.125088  18.0  53.0  60.0  66.0   97.0
Hypertension       9245.0  54.560411  15.102332   0.0  44.0  55.0  65.0  118.0
Obesity            9231.0  29.983750  20.065039 -47.0  16.0  30.0  43.0  107.0


Ahora cambiaremos la distribución de medicamentos por unos más coherentes.

In [26]:
# 1. Definir medicamentos más realistas y su frecuencia para cada condición
medications = {
    'Diabetes': (['Metformin', 'Insulin', 'Glipizide'], [0.2, 0.7, 0.1]),  # Ejemplo: ponderaciones para Diabetes
    'Arthritis': (['Methylprednisolone', 'Acetaminophen', 'Diclofenac'], [0.6, 0.1, 0.3]),  # Ejemplo: ponderaciones para Arthritis
    'Obesity': (['Semaglutide', 'Phentermine', 'Lorcaserin'], [0.5, 0.3, 0.2]),  # Ejemplo: ponderaciones para Obesity
    'Hypertension': (['Captopril', 'Nifedipine', 'Nitroglycerine'], [0.4, 0.35, 0.25]),  # Ejemplo: ponderaciones para Hypertension
    'Cancer': (['Paclitaxel', 'Cisplatin', 'Doxorubicin'], [0.4, 0.4, 0.2]),  # Ejemplo: ponderaciones para Cancer
    'Asthma': (['Albuterol', 'Fluticasone', 'Montelukast'], [0.4, 0.5, 0.1])  # Ejemplo: ponderaciones para Asthma
}

# 2. Reemplazar medicamentos en el DataFrame
for condition, (med_list, weights) in medications.items():
    # Filtrar pacientes con la condición actual
    condition_df = new_df[new_df['Medical Condition'] == condition]

    # Calcular el número de pacientes a reemplazar (por ejemplo, reemplazar el 50% de los medicamentos)
    num_to_replace = int(len(condition_df) * 0.8)  # Cambia el porcentaje según sea necesario

    # Seleccionar aleatoriamente pacientes para reemplazar
    indices_to_replace = np.random.choice(condition_df.index, num_to_replace, replace=False)

    # Generar medicamentos basados en las ponderaciones
    new_medications = np.random.choice(med_list, num_to_replace, p=weights)

    # Aplicar los nuevos valores de medicamento
    new_df.loc[indices_to_replace, 'Medication'] = new_medications


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>